In [1]:
#Import useful libraries
import json
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import GridSearchCV
from tensorflow.keras import utils
from LoadFaceData import read_path, resize_image, IMAGE_SIZE, load_dataset

In [2]:
#Define some useful variables
Ourfaceimage = []
Ourfacelabel = []
faceimage = []
frame_gray = []

#Read the face pictures from our path, form of list
faceimages, facelabel = read_path("D:\Limerick course year 4\AI & FYP\Conda Jupyter notebook\My AI Folder\FYP\Facial Recognization By SVM\Facial data")

#transfer the labels list in to data
facelabel = np.array(facelabel)

#Get the number of face, which is depended by number the pictures files
labels1 = list(set(facelabel))
face_num = len(labels1)

# Store the test name into a dic called constrast_table. 
num = [i for i in range(face_num)]
contrast_table = dict(zip(num, labels1))
with open('contrast_table', 'w') as f:
    f.write(json.dumps(contrast_table))

'''
# 样本集的形状
print(faces.images.shape)

faceimage = np.array(faceimage)
faceimage = faceimage.reshape(-1, IMAGE_SIZE*IMAGE_SIZE*3)

print(faceimage.shape)
print(facelabel)
print(facelabel.shape)'''

#Go through all images, transfer ot into grey picture.
for face_image in faceimages:
    frame_gray.append(cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY))

#Transfer all picture into array, and reshape the array into two Dim.ayyay content: [number of pictures, IMAGE_SIZE*IMAGE_SIZE] 
frame_gray = np.array(frame_gray)
face_image = frame_gray.reshape(-1, IMAGE_SIZE*IMAGE_SIZE)

#USing the split to seperate the raw data into train image, train labels, test images, testl labels.
xtrain, xtest, ytrain, ytest = train_test_split(face_image, facelabel,
                                                random_state=2)

#Pirnt out the shape of train set(images and labels) shape 
print(xtrain.shape)
print(ytrain.shape)

(4052, 4096)
(4052,)


In [3]:
# DO the PCA precess in this pictures, and keep 100 Dims
n_components =150

# Train the PCA
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True, random_state=42).fit(xtrain)

# Decease the dimention in these train sets.
xtrain_pca = pca.transform(xtrain)
xtest_pca = pca.transform(xtest)

print(xtest_pca.shape)
print(xtest.shape)

(1351, 150)
(1351, 4096)


In [4]:
#Training a Model

#Define some useful variables.
names = []

#Open the contrast_tabels, which is a dictionary, like: 0: "Jufeng Yang", 1: "Jiahao Wang".
with open('contrast_table', 'r') as f:
            contrast_table = json.loads(f.read())
#Instantial a SVM.Arguments is "RBF", C = 0.05 
svc = SVC(kernel='rbf',C=0.05)

# Train a SVM
svc.fit(xtrain_pca, ytrain)

#Save a  model, used to predict new pictures
joblib.dump(svc, "my_SVM_model.m")

#Predict based on the test dataset splits.
yfit = svc.predict(xtest_pca)
for i in range(face_num):
    names.append(contrast_table[str(i)])
names = np.array(names)

#Pirnt a summary report.
print(classification_report(ytest, yfit,
                            target_names=names))

              precision    recall  f1-score   support

 Zhongen Qin       1.00      0.99      0.99       256
 Jiahao Wang       0.55      1.00      0.71       390
     Yuan Li       1.00      0.79      0.88       252
 Xingda Zhou       1.00      0.02      0.03       115
 Jufeng Yang       1.00      0.25      0.40       192
     Yuxi Wu       1.00      0.93      0.96       146

    accuracy                           0.76      1351
   macro avg       0.92      0.66      0.66      1351
weighted avg       0.87      0.76      0.72      1351



In [6]:
# PCA processing of face data, retaining 100-dimensional features
#n_components =30
Pre_names = []
counter = 0
right_counter = 0
test_frame_gray = []
Predictresult = []
#Read the face pictures from our path
Testfaceimages, _= read_path("D:\Limerick course year 4\AI & FYP\Conda Jupyter notebook\My AI Folder\FYP\Facial Recognization By SVM\Test face\Jufeng Yang")

for face_image in Testfaceimages:
    test_frame_gray.append(cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY))
test_frame_gray = np.array(test_frame_gray)
Testfaceimage = test_frame_gray.reshape(-1, IMAGE_SIZE*IMAGE_SIZE)

#print(Testfaceimage.shape)

with open('contrast_table', 'r') as f:
            contrast_table = json.loads(f.read())

# Training for PCA
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True, random_state=42).fit(Testfaceimage)

Test_face = pca.transform(Testfaceimage)

load_SVM = joblib.load("my_SVM_model.m")

Predictresult = load_SVM.predict(Test_face)
#print(Predictresult)
for i in Predictresult:
    counter += 1
    if i == "Jufeng Yang":
        right_counter += 1
print("Accuracy:", right_counter/counter)        

Accuracy: 0.660480735242135
